In [ ]:
import os

import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
import pyspark.sql.functions as F
from google.cloud import bigquery

from config import ASSETS_TOPIC, CREDENTIALS_FILE, GCS_BUCKET, PROJECT_ID

In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = CREDENTIALS_FILE

pyspark_version = pyspark.__version__

In [ ]:
dataset_id = "assets_v2"  
table_id = "assets_all"


In [ ]:
conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('CryptoCurrencyConsumer') \
    .set("spark.jars", "/opt/homebrew/lib/gcs-connector-hadoop3-2.2.5.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", CREDENTIALS_FILE) \
    .set("spark.jars.packages", f"org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version},com.google.cloud.spark:spark-bigquery-with-dependencies_2.12:0.15.1-beta")

In [ ]:
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", CREDENTIALS_FILE)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

In [ ]:
spark = SparkSession \
    .builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [ ]:
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)


In [ ]:
assets_stream = spark \
    .read \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "assets-topic") \
    .option("startingOffsets", "earliest") \
    .load()

In [ ]:
from pyspark.sql import types

assets_spark_schema = types.StructType() \
    .add("id", types.StringType()) \
    .add("rank", types.StringType()) \
    .add("symbol", types.StringType()) \
    .add("name", types.StringType()) \
    .add("supply", types.StringType()) \
    .add("maxSupply", types.StringType()) \
    .add("marketCapUsd", types.StringType())\
    .add("priceUsd", types.StringType())\
    .add("changePercent24Hr", types.StringType())\
    .add("vwap24Hr", types.StringType())\
    .add("timestamp", types.StringType())

In [ ]:
from pyspark.sql import functions as F

assets_stream = assets_stream \
  .select(F.from_json(F.col("value").cast('STRING'),assets_spark_schema)).alias("assets") \
  .select("assets.from_json(CAST(value AS STRING)).*")

assets_stream_ts = assets_stream.withColumn('processing_timestamp', F.current_timestamp())

In [ ]:
assets_stream_ts.show(truncate=False)

In [ ]:
assets_stream_ts.write \
  .format("bigquery") \
    .option("table", f"{PROJECT_ID}.{dataset_id}.{table_id}") \
    .option("partitionField", "processing_timestamp")\
    .option("clusteredFields", "id")\
    .option("temporaryGcsBucket", GCS_BUCKET) \
    .mode("append") \
    .save()

In [ ]:
spark.stop()